## Overall Breakdown

1. Collect responses for each questions [done]
2. Parse data to call IBM Tone API
3. Display into the framework so user can get tone result for each question as well as the overall tone.
4. Add the tone labels back to the excell sheet.

## Bonus

* If TKH want us to add the feedback to their exell sheet we could use google doc API to do a GET and POST request.
* Overall rating column will be predicted through linear regression model.
* Revisit this problem using LNTK instead of IBM Tone API for sentiment analytics.

In [1]:
import pandas as pd

In [2]:
file = "Example feedback.xlsx"

In [3]:
df = pd.read_excel(file)

In [4]:
df.head()

,What’s your first name?,What’s your last name?,Thank you {{field:7b5d4741-fcb6-415e-b0c6-d6d3d813a76c}}. Please enter your email address.,What topic do you feel like you struggled with the most this week?,Please inform us of how we could make the aforementioned topic more understandable and beneficial to all program participants?,How would you rate this week's tech lab?,What is the reason for this rating?,How would you rate this week's tech project?,What is the reason for this rating?.1,How would you rate this week's career content?,...,"Are there any external factors (positively or negatively) affecting your performance, attendance, and/or overall progression in the program?","Please elaborate on the aforementioned external factors affecting your performance, attendance, and/or overall progression in the program. Let us know how we can help.",Are there any resources that aren't being provided by TKH that could assist you in successfully completing this program?,We're passionate about ensuring all of our participants are equipped with the necessary tools for success. Please inform us of how we could be of further assistance to you and your growth while in our program.,Additional comments and/or notes:,"Is there a fellow or TKH staff member you would like to shine a light on this week? If so, who are they and why? (you can also choose yourself if applicable)",Did you meet with a volunteer or mentor this week?,Submitted At,Token,Phase
0,Joe,Doe,example1@gmail.com,Deploying the heroku website,I think it might have been an issue with the w...,5,We got so much done during the lab time and we...,5,This alines with the lab since we only had tha...,5,...,No,NaN,No,NaN,I'm so excitedddd for phase two. Please let me...,Michael and Saraubh (two of the volunteers) re...,NaN,NaN,NaN,NaN
1,Anne,Frank,example2@gmail.com,Project,"Have the material, rubric and expectations wel...",5,NaN,5,NaN,5,...,No,NaN,No,NaN,NaN,Shoutouts to the amazing projects made by ever...,NaN,NaN,NaN,NaN
2,John,Doe,example3@gmail.com,the yield curve proje t,"more practice in class, more similar examples",5,it was good practice,4,hard but cool,5,...,Yes,kids at home are driving me nuts,"yes, a second monitor",I need another monitor,no,Nickala,"yes, my mentor",NaN,NaN,NaN
3,Edward,Brown,example4@gmail.com,the lab,we need to do pair programming more often in c...,4,it was alright,5,it was really helpful in terms of practicing w...,4,...,No,NaN,No,NaN,"You guys are so awesome, thank you for everyth...",My teammate Clariza,"yes, both",NaN,NaN,NaN
4,Maria,Rojas,example5@gmail.com,understanding the new machine learning model -...,we need to do more examples in class and durin...,4,"I liked it, a bit too hard though",5,nice portfolio piece,5,...,"yes, I have some ongoing health issues, possib...",surgery,No,NaN,No,John and Jane,No,NaN,NaN,NaN


In [5]:
columns = ["Please inform us of how we could make the aforementioned topic more understandable and beneficial to all program participants? ",\
       "What is the reason for this rating?",\
       "What is the reason for this rating?.1", 
       "What is the reason for this rating?.2",
       "What is the reason for this rating?.3",
       "What is the reason for this rating?.4", 
        "Please elaborate on the aforementioned external factors affecting your performance, attendance, and/or overall progression in the program. Let us know how we can help. ",
        "Additional comments and/or notes:",
        "Is there a fellow or TKH staff member you would like to shine a light on this week? If so, who are they and why? (you can also choose yourself if applicable)"]

In [6]:
import urllib
import requests
text = "Feel very rushed with the hackathon"

def get_tone(text):
    '''
    Get tone for text uses IBM tone API
    '''
    key = ("apikey", "dQLp8JO_J58ODhzuea9c9NMEk-3FloI-txKilaeU_goO")
    link = "https://api.us-south.tone-analyzer.watson.cloud.ibm.com/instances/a1c60bfe-61c6-49d1-a4d1-b435f853e9a5/v3/tone?"
    params = {"version":"2017-09-21","text":text}
    r = requests.get(link+urllib.parse.urlencode(params), auth=key)
    data = r.json()
    tone = ""
    for t in data["document_tone"]["tones"]:
        tone += f"{t['tone_name']} @ {t['score']} "
        
    return tone
    
get_tone(text)

'Sadness @ 0.545588 Confident @ 0.80026 '

In [7]:
def getting_text(columns):
    for col in columns:
        df[f'response+{col}'] = df[col].apply(get_tone)
    
results = getting_text(columns)

In [8]:
df

,What’s your first name?,What’s your last name?,Thank you {{field:7b5d4741-fcb6-415e-b0c6-d6d3d813a76c}}. Please enter your email address.,What topic do you feel like you struggled with the most this week?,Please inform us of how we could make the aforementioned topic more understandable and beneficial to all program participants?,How would you rate this week's tech lab?,What is the reason for this rating?,How would you rate this week's tech project?,What is the reason for this rating?.1,How would you rate this week's career content?,...,Phase,response+Please inform us of how we could make the aforementioned topic more understandable and beneficial to all program participants?,response+What is the reason for this rating?,response+What is the reason for this rating?.1,response+What is the reason for this rating?.2,response+What is the reason for this rating?.3,response+What is the reason for this rating?.4,"response+Please elaborate on the aforementioned external factors affecting your performance, attendance, and/or overall progression in the program. Let us know how we can help.",response+Additional comments and/or notes:,"response+Is there a fellow or TKH staff member you would like to shine a light on this week? If so, who are they and why? (you can also choose yourself if applicable)"
0,Joe,Doe,example1@gmail.com,Deploying the heroku website,I think it might have been an issue with the w...,5,We got so much done during the lab time and we...,5,This alines with the lab since we only had tha...,5,...,NaN,Analytical @ 0.910138 Tentative @ 0.991187,Joy @ 0.628821 Confident @ 0.709964 Analytical...,Joy @ 0.773552,Joy @ 0.920292 Confident @ 0.942754,Analytical @ 0.914626,Joy @ 0.673229 Analytical @ 0.830561,,Tentative @ 0.902459 Analytical @ 0.658877,Analytical @ 0.670204
1,Anne,Frank,example2@gmail.com,Project,"Have the material, rubric and expectations wel...",5,NaN,5,NaN,5,...,NaN,Analytical @ 0.90928,,,,,,,,Joy @ 0.923186
2,John,Doe,example3@gmail.com,the yield curve proje t,"more practice in class, more similar examples",5,it was good practice,4,hard but cool,5,...,NaN,Analytical @ 0.560098,Joy @ 0.827954,Joy @ 0.601882,Joy @ 0.789159 Confident @ 0.961633,Joy @ 0.761702 Analytical @ 0.579367,Analytical @ 0.997482,,,
3,Edward,Brown,example4@gmail.com,the lab,we need to do pair programming more often in c...,4,it was alright,5,it was really helpful in terms of practicing w...,4,...,NaN,,,Joy @ 0.545824 Analytical @ 0.955445,Sadness @ 0.720536,Tentative @ 0.786991,,,Joy @ 0.956088 Confident @ 0.937689,
4,Maria,Rojas,example5@gmail.com,understanding the new machine learning model -...,we need to do more examples in class and durin...,4,"I liked it, a bit too hard though",5,nice portfolio piece,5,...,NaN,Analytical @ 0.858259,Sadness @ 0.706107,Joy @ 0.732486 Analytical @ 0.955445,Joy @ 1.0,,Joy @ 0.924233,Sadness @ 0.916667 Fear @ 0.931034,,
5,Spongebob,Squarepants,example6@gmail.com,kmeans clustering,Perhaps breaking it down a bit more into manag...,5,"the lab was fair, with the right amount of cha...",5,It was appropriate for a phase 3 project.,3,...,NaN,Sadness @ 0.525603 Tentative @ 0.822231,Joy @ 0.566513 Confident @ 0.543112,,Joy @ 0.555217,Sadness @ 0.619474,Tentative @ 0.85813,,,
6,Jen,Mendez,example12@gmail.com,Implementing django with the different kinds o...,Follow on tutorials of django during class at ...,3,I like the implementation of SQL practice but ...,4,Enjoy building my portfolio but believe I have...,5,...,3.0,,Tentative @ 0.5538,Joy @ 0.69516 Tentative @ 0.786991,Joy @ 0.902093 Confident @ 0.86436,Sadness @ 0.59239 Tentative @ 0.615352,Joy @ 0.737562 Analytical @ 0.687768,,Joy @ 0.938827,Joy @ 0.737011 Analytical @ 0.762356
7,Mark,Carbajal,example7@gmail.com,Everything made sense this week,None,5,Everyhting was very clear and I was able to fo...,5,The project has given me the change to create ...,4,...,3.0,,Joy @ 0.710426 Confident @ 0.932155,,Joy @ 0.59329 Tentative @ 0.876534 Analytical ...,Analytical @ 0